In [2]:
!pip install pytorch_lightning
!pip install lightning
!pip install torch
!pip install torchvision
!pip install datasets
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Read the Config file from config.py

In [6]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from config import get_config
from lightning_dataset import BilingualLightning
from lightning_model import TransformerLightning
from lightning_train import train_transformer

### Edit the config file to accomodate the batch_size on available GPU machine

In [7]:
from config import get_config


In [8]:
cfg = get_config()

In [9]:
cfg

{'parameter_sharing': False,
 'd_ff': 2048,
 'batch_size': 32,
 'num_epochs': 30,
 'lr': 0.0001,
 'seq_len': 160,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': True,
 'tokenizer_file': 'tokenizer_{0}.json',
 'rundir': 'runs',
 'experiment_name': 'tmodel_dynamic_padding',
 'ds_mode': 'Online',
 'ds_path': None,
 'ds_name': 'opus_books',
 'save_ds_to_disk': True}

In [10]:
cfg['parameter_sharing'] = True
cfg['d_ff'] = 1024
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 40
cfg['ds_mode'] = 'Online'
cfg['lang_tgt'] = 'fr'
cfg['save_ds_to_disk'] = False
cfg['experiment_name'] = f"tmodel_dynamic_pad_150_tokens_batch_{cfg['batch_size']}_amp_dff{cfg['d_ff']}"

In [11]:
# updated config
cfg

{'parameter_sharing': True,
 'd_ff': 1024,
 'batch_size': 16,
 'num_epochs': 40,
 'lr': 0.0001,
 'seq_len': 160,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': 'tokenizer_{0}.json',
 'rundir': 'runs',
 'experiment_name': 'tmodel_dynamic_pad_150_tokens_batch_16_amp_dff1024',
 'ds_mode': 'Online',
 'ds_path': None,
 'ds_name': 'opus_books',
 'save_ds_to_disk': False}

### Enable the Mixed Precision Training

In [12]:
torch.cuda.amp.autocast(enabled=True)

In [13]:
torch.set_float32_matmul_precision('medium')

### Train the model

In [14]:
opus_fr = BilingualLightning(cfg)
opus_fr.prepare_data()
opus_fr.setup()
train_data_loader = opus_fr.train_dataloader()

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [15]:
tokenizer_src = opus_fr.tokenizer_src
tokenizer_tgt = opus_fr.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = TransformerLightning(cfg, loss_criterion, tokenizer_src, tokenizer_tgt, num_validation_examples=2, epochs = cfg['num_epochs'])
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)

scheduler = OneCycleLR(
    optimizer,
    max_lr=1E-3,
    steps_per_epoch=len(train_data_loader),
    epochs=cfg['num_epochs'],
    pct_start=5/cfg['num_epochs'],
    div_factor=10,
    three_phase=True,
    final_div_factor=10,
    anneal_strategy='linear'
)
transformer_model.set_scheduler_dict(scheduler, freq = 'step')

Total Parameters: 61847890


In [ ]:
train_transformer(transformer_model, opus_fr, cfg, epochs=cfg['num_epochs'])

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 61.8 M
------------------------------------------

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Listen!"
    TARGET: Écoutez…
 PREDICTED: !
*****************************************

*****************************************
    SOURCE: As the "Bonadventure" no longer existed, six months at least would be required for the construction of a new vessel.
    TARGET: Le Bonadventure n'existant plus, six mois, au moins, seraient nécessaires pour la construction d'un nouveau navire.
 PREDICTED: Quant à le marin , ne s ' est pas six mois , six mois de l ' heure d ' un vent .
*****************************************



/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


...

*****************************************
    SOURCE: The horror of death reanimated her,−−
    TARGET: L’horreur de la mort la ranima.
 PREDICTED: L ’ horreur de la mort lui :
*****************************************

*****************************************
    SOURCE: Going close by a bench on the breakwater he sat down, tired already ofwalking and out of humour with his stroll before he had taken it.
    TARGET: Comme il frôlait un banc sur le brise-lames, il s'assit, déjà las demarcher et dégoûté de sa promenade avant même de l'avoir faite.
 PREDICTED: Il se tenait à un banc , sur le brise - lames , déjà las et de l ' air d ' être mis à la promenade , avant qu ' il l ' eût .
*****************************************

Epoch 18: train_loss=1.8016


Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: A very good workman, he could speak well, put himself at the head of every opposition, and had at last become the chief of the discontented.
    TARGET: Tres bon ouvrier, il parlait bien, se mettait a la tete de toutes les réclamations, avait fini par etre le chef des mécontents.
 PREDICTED: Un bon ouvrier , on pouvait parler , se mettait a la tete , en tete de tous , en finit par faire de la .
*****************************************

*****************************************
    SOURCE: He redoubled his attention.
    TARGET: Il redoubla d’attention.
 PREDICTED: Il redoubla d ' attention .
*****************************************

Epoch 19: train_loss=2.0021


Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Sometimes bending over the forecastle railings, sometimes leaning against the sternrail, I eagerly scoured that cotton-colored wake that whitened the ocean as far as the eye could see!
    TARGET: Tantôt penché sur les bastingages du gaillard d'avant, tantôt appuyé à la lisse de l'arrière, je dévorais d'un oeil avide le cotonneux sillage qui blanchissait la mer jusqu'à perte de vue !
 PREDICTED: Parfois , par le gaillard des deux grilles , tantôt , je avec ardeur à l ' du haut que l ' Océan même pouvait voir !
*****************************************

*****************************************
    SOURCE: They had recognized Quasimodo.
    TARGET: On avait reconnu Quasimodo.
 PREDICTED: On avait reconnu Quasimodo .
*****************************************

